In [1]:
pip install Faker

     |████████████████████████████████| 1.2 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 16.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install jedi==0.17.2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import random
from datetime import date
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import os
from faker import Faker

In [2]:
fake = Faker()

In [3]:
def get_list_of_offices():
    return ["New York", "Austin", "Seattle","San Francisco"]

In [4]:
def get_list_of_titles():
    return ["Engineer","Sales Manager","Analyst","Sales Representative"]

In [5]:
def calculate_salary(title):
    if title=="Engineer" :
        return random.uniform(500,700)
    elif title=="Sales Manager":
        return random.uniform(800,900)
    elif title=="Analyst":
        return random.uniform(900,1000)
    elif title=="Sales Representative":
        return random.uniform(1000,1100)
    else:
        return 0

    
    
    
    

In [6]:
def create_customers_df(number_of_customers):
    customers_list=[]
    
    for i in range(number_of_customers):
        gender = np.random.choice(["Male","Female"],p=[0.5,0.5])
        birth_date=fake.date_between(start_date="-70y", end_date="-18y")
        start_date=fake.date_between(start_date=birth_date.replace(birth_date.year+18), end_date='today')
        title=np.random.choice(get_list_of_titles())
        first_name=fake.first_name_male() if gender=="Male"  else fake.first_name_female()
        last_name=fake.last_name()
        salary=round(calculate_salary(title),2)
        bonus_ratio=random.uniform(0.15,0.2)
        customers_list.append(
            {
                "customer_id":(i+1),
                "first_name":first_name,
                "last_name": last_name,
                "gender":gender,
                "birth_date": birth_date,
                "start_date": start_date,
                "office": np.random.choice(get_list_of_offices()),
                "title": title,
                "address": fake.address(),
                "salary": salary,
                "bonus":  round((salary * bonus_ratio/500) *500,2),
                "e_mail": (first_name+"."+last_name+"@"+fake.email().split("@")[1]).lower()
            }
        )
    return pd.DataFrame(customers_list)
        

In [7]:
customers_df=create_customers_df(10)

In [8]:
customers_df.head()

,customer_id,first_name,last_name,gender,birth_date,start_date,office,title,address,salary,bonus,e_mail
0,1,Amy,Pham,Female,1952-10-21,1999-10-12,Seattle,Engineer,"527 Andre Junctions\nNew Richard, WV 30108",636.08,120.83,amy.pham@example.org
1,2,Cameron,Chambers,Male,1996-08-09,2015-05-14,San Francisco,Engineer,"3741 Cameron Via Suite 843\nSouth Jose, ME 14710",589.30,92.86,cameron.chambers@example.com
2,3,Russell,Johnson,Male,1961-04-02,1987-01-02,Seattle,Engineer,"4878 Harrington Ridge\nSancheztown, CT 09520",610.30,97.17,russell.johnson@example.net
3,4,Brian,Valdez,Male,1976-08-12,2013-08-03,Austin,Engineer,"891 Gina Mission Suite 987\nMatthewville, OK 1...",522.70,88.79,brian.valdez@example.net
4,5,Elizabeth,Webb,Female,1952-02-12,2019-06-17,New York,Engineer,"957 Marquez Expressway\nJamesland, NE 61508",566.06,96.22,elizabeth.webb@example.com


In [10]:
customers_df.to_csv("customers.csv",index=False)

In [9]:
es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])

In [10]:
es

<Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])>

In [30]:
proba_doc={
    "autor":"Danilo",
    "tekst":"Random tekst",
    "datum":date.today()
}

In [31]:
proba_doc

{'autor': 'Danilo',
 'tekst': 'Random tekst',
 'datum': datetime.date(2021, 10, 14)}

In [32]:
red=es.index(index="proba",body=proba_doc,id=1)

<ipython-input-32-783339a4e9e4>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in 8.0.0. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  red=es.index(index="proba",body=proba_doc,id=1)
/Users/danilojanjusevic/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [12]:
request_body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "autocomplete_analyzer": {
                    "tokenizer": "autocomplete",
                    "filter": [
                        "lowercase"
                    ]
                },
                "autocomplete_search_analyzer": {
                    "tokenizer": "keyword",
                    "filter": [
                        "lowercase"
                    ]
                }
            },
            "tokenizer": {
                "autocomplete": {
                    "type": "edge_ngram",
                    "min_gram": 1,
                    "max_gram": 30,
                    "token_chars": [
                        "letter",
                        "digit",
                        "whitespace"
                    ]
                }
            }
        }
    },
    'mappings': {
        'doc': {
            'properties': {
                'customer_id':
                    {
                        "type": "integer",
                    },
                "first_name":
                    {
                        "type": "text"
                    },
                "last_name":
                    {
                        "type":"text"
                    },
                "gender":
                    {
                        "type":"text"
                    },
                "birth_date":
                    {
                        "type":"date",
                        "format": "yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis"
                    },
                "start_date":
                    {
                        "type":"date",
                        "format": "yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis"
                    },
                "office":
                    {
                        "type":"text"
                    },
                "title":
                    {
                        "type":"text"
                    },
                "address":
                    {
                        "type":"text"
                    },
                "salary":
                    {
                        "type":"double"
                    },
                "bonus":
                    {
                        "type":"double"
                    },
                "e_mail":
                    {
                        "type":"text"
                    }
                }
            }
        }
    }

In [13]:
name_of_index="customers"

In [14]:
res=es.index(index=name_of_index,body=request_body)

<ipython-input-14-8d6acd5af9d8>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in 8.0.0. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res=es.index(index=name_of_index,body=request_body)


In [19]:
class ElasticExporter:
    
    @staticmethod
    def export_dataframe(df,index_name):
        list_of_instances=[]
        for i in range(len(df)):
            row=df.loc[i]
            instance={}
            for j in range(len(row)):
                column=row[j]
                instance[df.columns[j]]=column
            op_dict={
                "index": {
                    "_index": index_name
                }
            }
            list_of_instances.append(op_dict)
            list_of_instances.append(instance)
        res=es.bulk(index=index_name,body=list_of_instances)
                
            
            
    
    
    
    
    

In [23]:
ElasticExporter.export_dataframe(customers_df,name_of_index)

In [24]:
es.search(index=name_of_index, body={"query": {"match_all": {}}})['hits']['total']

<ipython-input-24-226b95a426ba>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index=name_of_index, body={"query": {"match_all": {}}})['hits']['total']


{'value': 10, 'relation': 'eq'}

In [80]:
op_dict = {
    "index": {
             "_index": name_of_index,
             "_type": '_doc',
                    }
                }

In [74]:
new_dict={}

In [75]:
customers_df.columns[0]

'customer_id'

In [77]:
for i in range(len(customers_df.columns)):
    new_dict[customers_df.columns[i]]=customers_df.loc[0][i]
    

In [81]:
es.bulk(index=name_of_index,body=[op_dict,new_dict])

{'took': 188,
 'errors': False,
 'items': [{'index': {'_index': 'customers',
    '_type': '_doc',
    '_id': 'RRT0fnwB9UfO0s8KoMv7',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1,
    'status': 201}}]}

In [78]:
new_dict

{'customer_id': 1,
 'first_name': 'Steven',
 'last_name': 'Lopez',
 'gender': 'Male',
 'birth_date': datetime.date(1983, 11, 25),
 'start_date': datetime.date(2017, 8, 31),
 'office': 'New York',
 'title': 'Sales Representative',
 'address': '0241 Jacobson Drives Apt. 726\nSouth James, DC 83872',
 'salary': 1098.64,
 'bonus': 189.05,
 'e_mail': 'steven.lopez@example.net'}

In [27]:
query_body={
  "query": { 
    "bool": { 
      "must": [
        { "match": { "title":   "Engineer"        }}
      ]
    }
  }
}

In [28]:
es.search(index=name_of_index,body=query_body)

<ipython-input-28-f1ceba1006fa>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index=name_of_index,body=query_body)
/Users/danilojanjusevic/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 0.58093685,
  'hits': [{'_index': 'customers',
    '_type': '_doc',
    '_id': '6SuzgnwBoMiF0DNXZL8U',
    '_score': 0.58093685,
    '_source': {'customer_id': 1,
     'first_name': 'Amy',
     'last_name': 'Pham',
     'gender': 'Female',
     'birth_date': '1952-10-21',
     'start_date': '1999-10-12',
     'office': 'Seattle',
     'title': 'Engineer',
     'address': '527 Andre Junctions\nNew Richard, WV 30108',
     'salary': 636.08,
     'bonus': 120.83,
     'e_mail': 'amy.pham@example.org'}},
   {'_index': 'customers',
    '_type': '_doc',
    '_id': '6iuzgnwBoMiF0DNXZL8U',
    '_score': 0.58093685,
    '_source': {'customer_id': 2,
     'first_name': 'Cameron',
     'last_name': 'Chambers',
     'gender': 'Male',
     'birth_date': '1996-08-09',
     'start_date': '2015-05-14',
     'office': 'San Francisco',
 